In [1]:
# https://github.com/aladdinpersson/Machine-Learning-Collection/tree/master/ML/Pytorch/object_detection/YOLO

In [2]:
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT
from tqdm import tqdm
from torch.utils.data import DataLoader
import numpy as np
import time
import sys
sys.path.append('../')

from model import Yolov1
from dataset import VOCDataset, PapsDataset, train_transforms, val_transforms
from train import Compose, train_fn
from scheduler import CosineAnnealingWarmUpRestarts

from utils import (
    non_max_suppression,
    mean_average_precision,
    intersection_over_union,
    cellboxes_to_boxes,
    get_bboxes,
    plot_image,
    save_checkpoint,
    load_checkpoint,
)
from loss import YoloLoss

In [3]:
partition = np.load('../data/partition.npy', allow_pickle=True, encoding='latin1').item()
label = np.load('../data/labels_info.npy', allow_pickle=True, encoding='latin1').item()


In [4]:
train_dataset = PapsDataset(
    label,
#     partition['train'][0:100],
    partition['train'],
    transform=train_transforms,
)

test_dataset = PapsDataset(
    label,
#     partition['train'][0:100],
    partition['test'],
    transform=val_transforms,
)

In [5]:
# for image, matrix in train_dataset :
# #     print(image.shape)
# #     print(matrix.shape)
# #     print(path)
# #     print(boxes)
#     pass

In [6]:
GPU_NUM = 0 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
model = Yolov1(split_size=25, num_boxes=2, num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
loss_fn = YoloLoss()
# scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=10, T_mult=2, eta_max=0.0008,  T_up=5, gamma=0.5)

In [7]:
# model.load_state_dict(torch.load('../trained_model/cifar_net.pth'),strict=False)

In [8]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=12,
    num_workers=12,
#     pin_memory=False,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=12,
    num_workers=12,
    shuffle=False,
    drop_last=True,
    collate_fn=collate_fn
)
print(len(train_loader))
print(len(test_loader))

1248
312


In [9]:
# # image, label, bbox = next(iter(test_dataset))
# for images, labels, bboxes in train_loader :
# #     image, label, bbox = data
# #     print(images[0].shape)
#     c, h, w = images[0].shape
#     images = torch.cat(images).view(-1, c, h, w)
#     lc, lh, lw = labels[0].shape
#     labels = torch.cat(labels).view(-1, lc, lh, lw)
# #     print(images.shape)
# #     print((labels.shape))



In [10]:
# saved_dir = '../trained_model/'
# state = torch.load(saved_dir + 'epoch_' + str(12) +'_model.pt')
# epoch = state['epoch']
# model.load_state_dict(state['state_dict'], strict=False)
# optimizer.load_state_dict(state['optimizer'])
# scheduler.load_state_dict(state['scheduler'])

In [ ]:
epochs = 120
for epoch in range(0, epochs):
    stime = time.time()
    epoch_loss = train_fn(train_loader, model, optimizer, loss_fn)
#     scheduler.step()    
    mean_avg_prec = 0
    if epoch % 10 == 9 :
        pred_boxes, target_boxes = get_bboxes(
            test_loader, model, iou_threshold=0.4, threshold=0.4
        )

        mean_avg_prec = mean_average_precision(
            pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
        )
    print('{} epoch_loss {} mean_avg_prec {} time {} lr {}'.format(
            epoch, epoch_loss, mean_avg_prec, (time.time() - stime)/60, optimizer.param_groups[0]["lr"]))
    

  0%|          | 0/1248 [00:00<?, ?it/s]

0 epoch_loss (597.5217124865605, 366.9678306762989, 43.44648382908259, 143.04071557216156, 44.066683360399345) mean_avg_prec 0 time 7.832221178213755 lr 0.0005


  0%|          | 0/1248 [00:00<?, ?it/s]

1 epoch_loss (476.4904223466531, 305.69534182548523, 43.032542270727646, 89.00185124079387, 38.76068765307084) mean_avg_prec 0 time 7.90875198841095 lr 0.0005


  0%|          | 0/1248 [00:00<?, ?it/s]

...

  0%|          | 0/1248 [00:00<?, ?it/s]

58 epoch_loss (101.76932426752188, 45.17189234265914, 44.20336294938357, 7.971110387872427, 4.422958582777005) mean_avg_prec 0 time 7.799850122133891 lr 0.0005


  0%|          | 0/1248 [00:00<?, ?it/s]

59 epoch_loss (101.56400781105727, 44.71746767331393, 44.168905927966804, 8.058868008164259, 4.618766251411169) mean_avg_prec 0.041421547532081604 time 9.55809118350347 lr 0.0005


 43%|████▎     | 537/1248 [03:23<04:26,  2.67it/s, box_loss=49, cls_loss=3.77, loss=111, noob_loss=9.63, ob_loss=48.1]     

In [ ]:
pred_boxes, target_boxes = get_bboxes(
    test_loader, model, iou_threshold=0.4, threshold=0.4
)

mean_avg_prec = mean_average_precision(
    pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
)

In [ ]:
print(mean_avg_prec)

In [ ]:
# model.eval()

In [ ]:
# test_loader = DataLoader(
#     dataset=test_dataset,
#     batch_size=1,
#     num_workers=1,
#     shuffle=False,
#     drop_last=True,
# )
# pred_boxes, target_boxes = get_bboxes(
#     test_loader, model, iou_threshold=0.5, threshold=0.4
# )

# mean_avg_prec = mean_average_precision(
#     pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
# )

In [ ]:
saved_dir = '../trained_model/'
state = {
    'epoch': epochs,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
#     'scheduler' : scheduler.state_dict()
}

torch.save(state, saved_dir + 'epoch_' + str(90) +'_model.pt')

In [ ]:
def convert_cellboxes(predictions, S=23):
    """
    Converts bounding boxes output from Yolo with
    an image split size of S into entire image ratios
    rather than relative to cell ratios. Tried to do this
    vectorized, but this resulted in quite difficult to read
    code... Use as a black box? Or implement a more intuitive,
    using 2 for loops iterating range(S) and convert them one
    by one, resulting in a slower but more readable implementation.
    """

    predictions = predictions.to("cpu")
    batch_size = predictions.shape[0]
    print('batch_size', batch_size)
    predictions = predictions.reshape(batch_size, S, S, 11)
    bboxes1 = predictions[..., 2:6]
    bboxes2 = predictions[..., 7:11]
#     print('box1 {} box2 {}'.format(bboxes1, bboxes2))
    scores = torch.cat(
        (predictions[..., 1].unsqueeze(0), predictions[..., 6].unsqueeze(0)), dim=0
    )
    print('scores', scores.shape)
    best_box = scores.argmax(0).unsqueeze(-1)
#     print(best_box[0][0])
    best_boxes = bboxes1 * (1 - best_box) + best_box * bboxes2
    cell_indices = torch.arange(S).repeat(batch_size, S, 1).unsqueeze(-1)
    x = 1 / S * (best_boxes[..., :1] + cell_indices)
    y = 1 / S * (best_boxes[..., 1:2] + cell_indices.permute(0, 2, 1, 3))
    w_y = 1 / S * best_boxes[..., 2:4]
    converted_bboxes = torch.cat((x, y, w_y), dim=-1)
    predicted_class = predictions[..., :1].argmax(-1).unsqueeze(-1)
    best_confidence = torch.max(predictions[..., 1], predictions[..., 6]).unsqueeze(
        -1
    )
    converted_preds = torch.cat(
        (predicted_class, best_confidence, converted_bboxes), dim=-1
    )

    return converted_preds


def cellboxes_to_boxes(out, S=23):
    print('out', out.shape)
    converted_pred = convert_cellboxes(out).reshape(out.shape[0], S * S, -1)
    print('converted_pred', converted_pred.shape)
    converted_pred[..., 0] = converted_pred[..., 0].long()
    all_bboxes = []

    for ex_idx in range(out.shape[0]):
        bboxes = []

        for bbox_idx in range(S * S):
            bboxes.append([x.item() for x in converted_pred[ex_idx, bbox_idx, :]])
        all_bboxes.append(bboxes)

    return all_bboxes


In [ ]:
image, label, bboxes = next(iter(test_dataset))
image = image.unsqueeze(dim=0).to(device)
label = label.unsqueeze(dim=0)
print(image.shape)
model.eval()
with torch.no_grad():
    prediction = model(image)
    print('prediction',prediction.shape)
print('label', label.shape)    
true_bbox = cellboxes_to_boxes(label)
print('true_bbox', true_bbox[0][0])
bbox = cellboxes_to_boxes(prediction)
# print(bboxes)
print(len(true_bbox[0][0]))
print(len(bboxes))

